In [1]:
import swisseph as swe
from datetime import datetime
from typing import Dict, Any, Optional
import pytz

# Zodiac signs
SIGNS = [
    'Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo',
    'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces'
]

# Planet constants and names
PLANETS = [
    (swe.SUN, 'Sun'),
    (swe.MOON, 'Moon'),
    (swe.MERCURY, 'Mercury'),
    (swe.VENUS, 'Venus'),
    (swe.MARS, 'Mars'),
    (swe.JUPITER, 'Jupiter'),
    (swe.SATURN, 'Saturn'),
    (swe.URANUS, 'Uranus'),
    (swe.NEPTUNE, 'Neptune'),
    (swe.PLUTO, 'Pluto'),
    (swe.MEAN_NODE, 'Rahu'),
]


def get_planetary_positions(
    local_datetime: datetime,
    latitude: float,
    longitude: float,
    timezone: str = 'UTC',
    sidereal: bool = True,
    ayanamsa: int = swe.SIDM_LAHIRI
) -> Dict[str, Any]:
    """
    Calculate planetary positions for a given date, time, and location.
    
    Args:
        local_datetime: Local date and time (naive datetime)
        latitude: Location latitude (positive for North, negative for South)
        longitude: Location longitude (positive for East, negative for West)
        timezone: Timezone string (e.g., 'Asia/Kolkata', 'America/New_York')
        sidereal: If True, use sidereal zodiac (Vedic). If False, use tropical (Western)
        ayanamsa: Ayanamsa type for sidereal calculation (default: Lahiri)
    
    Returns:
        Dictionary containing datetime info, location, ayanamsa, ascendant, and planetary positions
    """
    
    # Convert local time to UTC
    local_tz = pytz.timezone(timezone)
    if local_datetime.tzinfo is None:
        local_dt = local_tz.localize(local_datetime)
    else:
        local_dt = local_datetime
    utc_dt = local_dt.astimezone(pytz.UTC)
    
    # Calculate Julian Day
    hour_decimal = utc_dt.hour + utc_dt.minute / 60 + utc_dt.second / 3600
    julian_day = swe.julday(utc_dt.year, utc_dt.month, utc_dt.day, hour_decimal)
    
    # Set sidereal mode if required
    if sidereal:
        swe.set_sid_mode(ayanamsa)
        calc_flag = swe.FLG_SIDEREAL
        ayanamsa_value = swe.get_ayanamsa(julian_day)
        zodiac_system = 'Sidereal'
    else:
        calc_flag = 0
        ayanamsa_value = 0.0
        zodiac_system = 'Tropical'
    
    # Initialize result dictionary
    result = {
        'input': {
            'local_datetime': local_datetime.strftime('%Y-%m-%d %H:%M:%S'),
            'timezone': timezone,
            'latitude': latitude,
            'longitude': longitude,
        },
        'calculated': {
            'utc_datetime': utc_dt.strftime('%Y-%m-%d %H:%M:%S'),
            'julian_day': julian_day,
            'zodiac_system': zodiac_system,
            'ayanamsa': {
                'name': 'Lahiri' if ayanamsa == swe.SIDM_LAHIRI else 'Other',
                'value': round(ayanamsa_value, 4)
            } if sidereal else None,
        },
        'ascendant': {},
        'planets': {}
    }
    
    # Calculate Ascendant (Lagna)
    try:
        houses = swe.houses(julian_day, latitude, longitude, b'P')  # Placidus house system
        ascendant_longitude = houses[1][0]  # Ascendant is first element
        
        if sidereal:
            ascendant_longitude = (ascendant_longitude - ayanamsa_value) % 360
        
        result['ascendant'] = _format_position(ascendant_longitude, 'Ascendant')
    except Exception as e:
        result['ascendant'] = {'error': str(e)}
    
    # Calculate planetary positions
    for planet_id, planet_name in PLANETS:
        try:
            calc_result = swe.calc_ut(julian_day, planet_id, calc_flag)
            lon = calc_result[0][0]
            speed = calc_result[0][3]  # Daily speed for retrograde check
            
            planet_data = _format_position(lon, planet_name)
            planet_data['speed'] = round(speed, 4)
            planet_data['retrograde'] = speed < 0
            
            result['planets'][planet_name] = planet_data
            
            # Calculate Ketu (opposite to Rahu)
            if planet_id == swe.MEAN_NODE:
                ketu_lon = (lon + 180) % 360
                ketu_data = _format_position(ketu_lon, 'Ketu')
                ketu_data['speed'] = round(speed, 4)
                ketu_data['retrograde'] = True  # Nodes are always retrograde
                result['planets']['Ketu'] = ketu_data
                
        except Exception as e:
            result['planets'][planet_name] = {'error': str(e)}
    
    return result


def _format_position(longitude: float, body_name: str) -> Dict[str, Any]:
    """Format longitude into zodiac position details."""
    sign_num = int(longitude // 30)
    degree_in_sign = longitude % 30
    degrees = int(degree_in_sign)
    minutes = int((degree_in_sign - degrees) * 60)
    seconds = int(((degree_in_sign - degrees) * 60 - minutes) * 60)
    
    return {
        'longitude': round(longitude, 4),
        'sign': SIGNS[sign_num],
        'sign_num': sign_num + 1,  # 1-indexed (Aries = 1)
        'degrees': degrees,
        'minutes': minutes,
        'seconds': seconds,
        'formatted': f"{degrees}° {minutes}' {seconds}\" {SIGNS[sign_num]}"
    }


def print_positions(positions: Dict[str, Any]) -> None:
    """Pretty print the planetary positions."""
    print("\n" + "=" * 70)
    print("PLANETARY POSITIONS")
    print("=" * 70)
    
    # Input details
    inp = positions['input']
    calc = positions['calculated']
    print(f"\nLocal Time:  {inp['local_datetime']} ({inp['timezone']})")
    print(f"UTC Time:    {calc['utc_datetime']}")
    print(f"Location:    {inp['latitude']}° N, {inp['longitude']}° E")
    print(f"Julian Day:  {calc['julian_day']}")
    print(f"Zodiac:      {calc['zodiac_system']}")
    
    if calc['ayanamsa']:
        print(f"Ayanamsa:    {calc['ayanamsa']['name']} ({calc['ayanamsa']['value']}°)")
    
    # Ascendant
    print("\n" + "-" * 70)
    asc = positions['ascendant']
    if 'error' not in asc:
        print(f"{'ASCENDANT (Lagna)':<20} {asc['formatted']:<25} {asc['longitude']:>10.4f}°")
    print("-" * 70)
    
    # Planets
    print(f"{'Planet':<20} {'Position':<25} {'Longitude':>10}  {'Retro'}")
    print("-" * 70)
    
    for planet_name, data in positions['planets'].items():
        if 'error' not in data:
            retro = 'R' if data['retrograde'] else ''
            print(f"{planet_name:<20} {data['formatted']:<25} {data['longitude']:>10.4f}°  {retro}")
    
    print("=" * 70)


# Example usage
if __name__ == "__main__":
    # Install required packages:
    # pip install pyswisseph pytz
    
    # Example: Current time in Hyderabad, India
    local_dt = datetime(2025, 6, 15, 10, 30, 0)  # June 15, 2025, 10:30 AM
    
    # Hyderabad coordinates
    lat = 17.3850
    lon = 78.4867
    tz = 'Asia/Kolkata'
    
    # Get positions (Sidereal/Vedic)
    positions = get_planetary_positions(
        local_datetime=local_dt,
        latitude=lat,
        longitude=lon,
        timezone=tz,
        sidereal=True
    )
    
    # Print formatted output
    print_positions(positions)
    
    # Access individual values from the map
    print("\n--- Accessing individual values ---")
    print(f"Moon Sign: {positions['planets']['Moon']['sign']}")
    print(f"Moon Degrees: {positions['planets']['Moon']['degrees']}° {positions['planets']['Moon']['minutes']}'")
    print(f"Ascendant: {positions['ascendant']['sign']}")
    print(f"Is Saturn Retrograde? {positions['planets']['Saturn']['retrograde']}")


PLANETARY POSITIONS

Local Time:  2025-06-15 10:30:00 (Asia/Kolkata)
UTC Time:    2025-06-15 05:00:00
Location:    17.385° N, 78.4867° E
Julian Day:  2460841.7083333335
Zodiac:      Sidereal
Ayanamsa:    Lahiri (24.2127°)

----------------------------------------------------------------------
ASCENDANT (Lagna)    4° 38' 45" Leo              124.6460°
----------------------------------------------------------------------
Planet               Position                   Longitude  Retro
----------------------------------------------------------------------
Sun                  0° 8' 58" Gemini             60.1496°  
Moon                 15° 27' 33" Capricorn       285.4594°  
Mercury              17° 45' 14" Gemini           77.7539°  
Venus                14° 56' 18" Aries            14.9385°  
Mars                 4° 35' 15" Leo              124.5876°  
Jupiter              6° 59' 55" Gemini            66.9988°  
Saturn               7° 4' 17" Pisces            337.0717°  
Uranus      

In [8]:
import swisseph as swe
from datetime import datetime
from typing import Dict, Any, Optional
import pytz

# current date and time
now = datetime.now()
print("Current date and time:", now)

# New York City coordinates:
# Convert UTC to local time
utc_time = positions['calculated']['utc_datetime']
utc_dt = datetime.strptime(utc_time, '%Y-%m-%d %H:%M:%S')
utc_dt = pytz.UTC.localize(utc_dt)
local_time = utc_dt.astimezone(pytz.timezone(tz))
print("UTC time:", utc_time)
print("Local time:", local_time.strftime('%Y-%m-%d %H:%M:%S'))
# 40.7069° N, 74.0113° W

lat = 40.7069
lon = -74.0113
tz = 'America/New_York'

positions = get_planetary_positions(
    local_datetime=local_time,
    latitude=lat,
    longitude=lon,
    timezone=tz,
    sidereal=True
)

print_positions(positions)



Current date and time: 2026-01-03 21:48:14.289692
UTC time: 2026-01-04 02:45:55
Local time: 2026-01-03 21:45:55

PLANETARY POSITIONS

Local Time:  2026-01-03 21:45:55 (America/New_York)
UTC Time:    2026-01-04 02:45:55
Location:    40.7069° N, -74.0113° E
Julian Day:  2461044.6152199074
Zodiac:      Sidereal
Ayanamsa:    Lahiri (24.2204°)

----------------------------------------------------------------------
ASCENDANT (Lagna)    20° 52' 31" Leo             140.8753°
----------------------------------------------------------------------
Planet               Position                   Longitude  Retro
----------------------------------------------------------------------
Sun                  19° 31' 13" Sagittarius     259.5204°  
Moon                 29° 6' 38" Gemini            89.1107°  
Mercury              9° 12' 36" Sagittarius      249.2102°  
Venus                18° 54' 14" Sagittarius     258.9040°  
Mars                 20° 51' 23" Sagittarius     260.8566°  
Jupiter         

In [9]:
positions

{'input': {'local_datetime': '2026-01-03 21:45:55',
  'timezone': 'America/New_York',
  'latitude': 40.7069,
  'longitude': -74.0113},
 'calculated': {'utc_datetime': '2026-01-04 02:45:55',
  'julian_day': 2461044.6152199074,
  'zodiac_system': 'Sidereal',
  'ayanamsa': {'name': 'Lahiri', 'value': 24.2204}},
 'ascendant': {'longitude': 140.8753,
  'sign': 'Leo',
  'sign_num': 5,
  'degrees': 20,
  'minutes': 52,
  'seconds': 31,
  'formatted': '20° 52\' 31" Leo'},
 'planets': {'Sun': {'longitude': 259.5204,
   'sign': 'Sagittarius',
   'sign_num': 9,
   'degrees': 19,
   'minutes': 31,
   'seconds': 13,
   'formatted': '19° 31\' 13" Sagittarius',
   'speed': 0.0,
   'retrograde': False},
  'Moon': {'longitude': 89.1107,
   'sign': 'Gemini',
   'sign_num': 3,
   'degrees': 29,
   'minutes': 6,
   'seconds': 38,
   'formatted': '29° 6\' 38" Gemini',
   'speed': 0.0,
   'retrograde': False},
  'Mercury': {'longitude': 249.2102,
   'sign': 'Sagittarius',
   'sign_num': 9,
   'degrees': 9,
